In [1]:
!git clone https://github.com/AlexandreNap/surf-crowd --recurse-submodules

Cloning into 'surf-crowd'...
remote: Enumerating objects: 396, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 396 (delta 55), reused 96 (delta 37), pack-reused 256
Receiving objects: 100% (396/396), 59.44 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Submodule 'surfers_detection/yolov5' (https://github.com/AlexandreNap/yolov5.git) registered for path 'surfers_detection/yolov5'
Cloning into '/kaggle/working/surf-crowd/surfers_detection/yolov5'...
remote: Enumerating objects: 12330, done.        
remote: Total 12330 (delta 0), reused 0 (delta 0), pack-reused 12330        
Receiving objects: 100% (12330/12330), 12.09 MiB | 17.44 MiB/s, done.
Resolving deltas: 100% (8462/8462), done.
Submodule path 'surfers_detection/yolov5': checked out 'c5f15168ee73fef02357fb147b6d1e55da14268e'


In [2]:
!pip install -r ./surf-crowd/surfers_detection/yolov5/requirements.txt

In [3]:
cp ../input/model-test/yolov5l_k32_best.pt ./surf-crowd/surfers_detection/yolov5/yolo_model.pt

In [4]:
mkdir data

In [5]:
import os
from shutil import move
from glob import iglob
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# The .py file has to be on the same directory as the folders containing the files!
root = Path("../input/spots-raw-images-to-2022-05-11")

# Using threading in case the operation becomes I/O bound (many files)
with ThreadPoolExecutor() as executor:
    for file in tqdm(iglob(str(root / "spots" /"**" / "*"))):
        file = Path(file)

        # The new filename is the name of the directory, and the suffix(es) of the original file
        new_filename = f"spot_{file.parent.name}_date_{file.name}"

        # Move AND rename simultaneosly
        executor.submit(move, file, Path("./data") / new_filename)

33448it [00:02, 12648.20it/s]


In [6]:
ls surf-crowd/surfers_detection/yolov5

CONTRIBUTING.md  data/       requirements.txt*      utils/
Dockerfile       detect.py   setup.cfg              val.py
LICENSE          export.py   template_no_auth.yaml  yolo_model.pt
README.md        hubconf.py  train.py
aws_lambda/      models/     tutorial.ipynb


In [7]:
[1,2][1]

2

In [8]:
import torch
import glob
from PIL import Image
import pandas as pd
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

files = [f for f in glob.glob("data/*.jpg", recursive=True) if os.path.getsize(f) > 0]
bs = 32
model = torch.hub.load('surf-crowd/surfers_detection/yolov5', 'custom', path='surf-crowd/surfers_detection/yolov5/yolo_model.pt', source='local')
model.conf = 0.4  # NMS confidence threshold
        
save_every = 2
all_preds = None
for i in range(1 + int(len(files) / bs)):
    batch_files = [f for f in files[i * bs : np.min([(i + 1) * bs, len(files) - 1])]]
    
    imgs = [Image.open(f).resize((1920, 1080)) for f in batch_files]
            
    results = model(imgs, size=1920)  # includes NMS
                   
    for j, pred in enumerate(results.pandas().xyxy):
        pred['file'] = batch_files[j].replace("data/", "")
        if all_preds is None:
            all_preds = pred
        else:
            all_preds = pd.concat([all_preds, pred])
    
    if i % save_every == 0:
        all_preds.to_csv("all_preds.csv", index=False)
                    
    results.save()

all_preds.to_csv("all_preds.csv", index=False)

YOLOv5 🚀 2022-10-17 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 367 layers, 46108278 parameters, 0 gradients
Adding AutoShape... 
Saved 32 images to runs/detect/exp
Saved 32 images to runs/detect/exp2
Saved 32 images to runs/detect/exp3
Saved 32 images to runs/detect/exp4
Saved 32 images to runs/detect/exp5
Saved 32 images to runs/detect/exp6
Saved 32 images to runs/detect/exp7
Saved 32 images to runs/detect/exp8
Saved 32 images to runs/detect/exp9
Saved 32 images to runs/detect/exp10
Saved 32 images to runs/detect/exp11
Saved 32 images to runs/detect/exp12
Saved 32 images to runs/detect/exp13
Saved 32 images to runs/detect/exp14
Saved 32 images to runs/detect/exp15
Saved 32 images to runs/detect/exp16
Saved 32 images to runs/detect/exp17
Saved 32 images to runs/detect/exp18
Saved 32 images to runs/detect/exp19
Saved 32 images to runs/detect/exp20
Saved 32 images to runs/detect/exp21
Saved 32 images to runs/detect/exp22
Saved 32 

In [9]:
all_preds

,xmin,ymin,xmax,ymax,confidence,class,name,file
0,1117.850464,612.980103,1141.436646,635.672485,0.926877,0,surfer,spot_Lacanau_date_2022-03-20_17-00.jpg
1,1789.491699,543.309448,1815.154297,557.544434,0.881262,0,surfer,spot_Lacanau_date_2022-03-20_17-00.jpg
2,1454.049561,562.768005,1467.720947,576.358582,0.842756,0,surfer,spot_Lacanau_date_2022-03-20_17-00.jpg
3,1638.487183,595.163086,1651.055786,605.342773,0.831200,0,surfer,spot_Lacanau_date_2022-03-20_17-00.jpg
4,611.197510,444.098755,623.041382,453.749329,0.816187,0,surfer,spot_Lacanau_date_2022-03-20_17-00.jpg
...,...,...,...,...,...,...,...,...
0,349.449951,761.364624,359.308594,766.085083,0.856159,0,surfer,spot_Anglet_GrandePlage_date_2022-04-29_10-45.jpg
1,204.877975,746.035095,216.595047,750.319519,0.837040,0,surfer,spot_Anglet_GrandePlage_date_2022-04-29_10-45.jpg
2,167.953659,745.361389,174.759232,750.534485,0.791105,0,surfer,spot_Anglet_GrandePlage_date_2022-04-29_10-45.jpg
3,145.168777,746.099182,151.190353,751.304504,0.736437,0,surfer,spot_Anglet_GrandePlage_date_2022-04-29_10-45.jpg


In [10]:
mkdir all_results

In [11]:
# The .py file has to be on the same directory as the folders containing the files!
root = Path("./runs/detect/")

# Using threading in case the operation becomes I/O bound (many files)
with ThreadPoolExecutor() as executor:
    for file in tqdm(iglob(str(root / "*" / "*"))):
        file = Path(file)
        # The new filename is the name of the directory, and the suffix(es) of the original file
        new_filename = f"{file.parent.name}_{file.name}"

        # Move AND rename simultaneosly
        executor.submit(move, file, Path("./all_results") / new_filename)

33447it [00:03, 10228.70it/s]


In [12]:
ls all_results

exp1000_image0.jpg   exp296_image11.jpg  exp530_image20.jpg  exp766_image3.jpg
exp1000_image1.jpg   exp296_image12.jpg  exp530_image21.jpg  exp766_image30.jpg
exp1000_image10.jpg  exp296_image13.jpg  exp530_image22.jpg  exp766_image31.jpg
exp1000_image11.jpg  exp296_image14.jpg  exp530_image23.jpg  exp766_image4.jpg
exp1000_image12.jpg  exp296_image15.jpg  exp530_image24.jpg  exp766_image5.jpg
exp1000_image13.jpg  exp296_image16.jpg  exp530_image25.jpg  exp766_image6.jpg
exp1000_image14.jpg  exp296_image17.jpg  exp530_image26.jpg  exp766_image7.jpg
exp1000_image15.jpg  exp296_image18.jpg  exp530_image27.jpg  exp766_image8.jpg
exp1000_image16.jpg  exp296_image19.jpg  exp530_image28.jpg  exp766_image9.jpg
exp1000_image17.jpg  exp296_image2.jpg   exp530_image29.jpg  exp767_image0.jpg
exp1000_image18.jpg  exp296_image20.jpg  exp530_image3.jpg   exp767_image1.jpg
exp1000_image19.jpg  exp296_image21.jpg  exp530_image30.jpg  exp767_image10.jpg
exp1000_image2.jpg   exp296_image22.jpg  exp530_i

In [13]:
!zip -r results_all.zip all_results/

  adding: all_results/ (stored 0%)
  adding: all_results/exp156_image0.jpg (deflated 5%)
  adding: all_results/exp753_image5.jpg (deflated 1%)
  adding: all_results/exp44_image6.jpg (deflated 4%)
  adding: all_results/exp108_image8.jpg (deflated 3%)
  adding: all_results/exp185_image26.jpg (deflated 10%)
  adding: all_results/exp191_image25.jpg (deflated 20%)
  adding: all_results/exp1011_image28.jpg (deflated 9%)
  adding: all_results/exp995_image7.jpg (deflated 6%)
  adding: all_results/exp115_image3.jpg (deflated 6%)
  adding: all_results/exp389_image11.jpg (deflated 8%)
  adding: all_results/exp104_image9.jpg (deflated 8%)
  adding: all_results/exp375_image1.jpg (deflated 6%)
  adding: all_results/exp906_image9.jpg (deflated 12%)
  adding: all_results/exp530_image31.jpg (deflated 8%)
  adding: all_results/exp771_image1.jpg (deflated 10%)
  adding: all_results/exp879_image10.jpg (deflated 11%)
  adding: all_results/exp878_image26.jpg (deflated 10%)
  adding: all_results/exp1001_imag

In [14]:
"""
import IPython
from IPython.display import display
IPython.display.Image(filename='./all_results/exp3_image4.jpg') 
"""

"\nimport IPython\nfrom IPython.display import display\nIPython.display.Image(filename='./all_results/exp3_image4.jpg') \n"

In [15]:
"""
import IPython
from IPython.display import display
IPython.display.Image(filename='./runs/detect/exp38/image8.jpg') 
"""

"\nimport IPython\nfrom IPython.display import display\nIPython.display.Image(filename='./runs/detect/exp38/image8.jpg') \n"

In [16]:
#ls runs/detect/exp10

In [17]:
"""
%%time
!python ./surf-crowd/surfers_detection/yolov5/detect.py --imgsz 1080 1920 \
--conf-thres 0.4 \
--weights ./surf-crowd/surfers_detection/yolov5/yolo_model.pt\
--line-thickness 1\
--source ./data \
--save-txt
"""

'\n%%time\n!python ./surf-crowd/surfers_detection/yolov5/detect.py --imgsz 1080 1920 --conf-thres 0.4 --weights ./surf-crowd/surfers_detection/yolov5/yolo_model.pt--line-thickness 1--source ./data --save-txt\n'

In [18]:
"""
with open('./surf-crowd/surfers_detection/yolov5/runs/detect/exp/labels/spot_Lacanau_date_2022-03-20_14-00.txt') as f:
    res = f.read()
print(res)
"""

"\nwith open('./surf-crowd/surfers_detection/yolov5/runs/detect/exp/labels/spot_Lacanau_date_2022-03-20_14-00.txt') as f:\n    res = f.read()\nprint(res)\n"

In [19]:
# !zip -r results_all.zip ./surf-crowd/surfers_detection/yolov5/runs/detect/exp/

In [20]:
#!zip -r results_labels.zip ./surf-crowd/surfers_detection/yolov5/runs/detect/exp/labels/

In [21]:
#ls

In [22]:
#IPython.display.Image(filename='./surf-crowd/surfers_detection/yolov5/runs/detect/exp3/images3.jpg') 